## Uvod

V tej lekciji bomo obravnavali:
- Kaj je klic funkcije in kje ga uporabljamo
- Kako ustvariti klic funkcije z OpenAI
- Kako vključiti klic funkcije v aplikacijo

## Cilji učenja

Po zaključku te lekcije boste znali in razumeli:

- Namen uporabe klica funkcije
- Nastavitev klica funkcije z uporabo storitve OpenAI
- Oblikovanje učinkovitih klicev funkcij za potrebe vaše aplikacije


## Razumevanje klicev funkcij

V tej lekciji želimo razviti funkcijo za naš izobraževalni startup, ki uporabnikom omogoča, da s pomočjo klepetalnega robota poiščejo tehnične tečaje. Priporočili bomo tečaje, ki ustrezajo njihovi ravni znanja, trenutni vlogi in tehnologiji, ki jih zanima.

Za izvedbo bomo uporabili kombinacijo:
 - `OpenAI` za ustvarjanje klepetalne izkušnje za uporabnika
 - `Microsoft Learn Catalog API` za pomoč uporabnikom pri iskanju tečajev glede na njihove zahteve
 - `Function Calling` za sprejem uporabniškega vprašanja in pošiljanje funkciji, ki izvede zahtevo na API.

Za začetek si poglejmo, zakaj sploh želimo uporabiti klic funkcije:

print("Sporočila v naslednji zahtevi:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # pridobimo nov odgovor od GPT, kjer lahko vidi odgovor funkcije


print(second_response.choices[0].message)


### Zakaj klicanje funkcij

Če ste opravili katero koli drugo lekcijo v tem tečaju, verjetno že razumete, kako zmogljivi so veliki jezikovni modeli (LLM). Upamo, da ste opazili tudi nekatere njihove omejitve.

Klicanje funkcij je funkcionalnost storitve OpenAI, ki je namenjena reševanju naslednjih izzivov:

Nedosledno oblikovanje odgovorov:
- Pred uvedbo klicanja funkcij so bili odgovori velikih jezikovnih modelov neurejeni in nedosledni. Razvijalci so morali pisati zapleteno kodo za preverjanje, da so lahko obravnavali vsako variacijo izhoda.

Omejena povezava z zunanjimi podatki:
- Pred to funkcijo je bilo težko vključiti podatke iz drugih delov aplikacije v kontekst pogovora.

S standardizacijo oblikovanja odgovorov in enostavno povezavo z zunanjimi podatki klicanje funkcij poenostavi razvoj in zmanjša potrebo po dodatni logiki za preverjanje.

Uporabniki niso mogli dobiti odgovorov, kot je "Kakšno je trenutno vreme v Stockholmu?". To je zato, ker so bili modeli omejeni na čas, ko so bili podatki naučeni.

Poglejmo si spodnji primer, ki ponazarja to težavo:

Recimo, da želimo ustvariti bazo podatkov o študentih, da bi jim lahko predlagali ustrezen tečaj. Spodaj sta dva opisa študentov, ki sta si po podatkih zelo podobna.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finshing his studies."

Želimo to poslati LLM-u, da obdela podatke. To lahko kasneje uporabimo v naši aplikaciji, da to pošljemo API-ju ali shranimo v bazo podatkov.

Ustvarimo dva enaka poziva, s katerima LLM-u natančno povemo, katere informacije nas zanimajo:


Želimo to poslati LLM-ju, da razčleni dele, ki so pomembni za naš izdelek. Tako lahko ustvarimo dva enaka poziva za navodila LLM-ju:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Po ustvarjanju teh dveh pozivov jih bomo poslali LLM z uporabo `openai.ChatCompletion`. Poziv shranimo v spremenljivko `messages` in vlogi dodelimo `user`. To je zato, da posnemamo sporočilo uporabnika, ki je napisano klepetalnemu robotu.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

Zdaj lahko pošljemo obe zahtevi LLM-ju in preučimo odziv, ki ga prejmemo.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Čeprav so pozivi enaki in so opisi podobni, lahko dobimo različne formate lastnosti `Grades`.

Če zgornjo celico zaženete večkrat, je lahko format `3.7` ali `3.7 GPA`.

To se zgodi zato, ker LLM prejme neorganizirane podatke v obliki napisanega poziva in prav tako vrne neorganizirane podatke. Potrebujemo strukturiran format, da vemo, kaj pričakovati pri shranjevanju ali uporabi teh podatkov.

Z uporabo funkcijskega klicanja lahko zagotovimo, da prejmemo nazaj strukturirane podatke. Pri uporabi funkcijskega klicanja LLM dejansko ne kliče ali izvaja nobenih funkcij. Namesto tega ustvarimo strukturo, ki ji LLM sledi pri svojih odgovorih. Te strukturirane odgovore nato uporabimo, da vemo, katero funkcijo naj zaženemo v naših aplikacijah.


![Diagram poteka klicanja funkcije](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.sl.png)


### Primeri uporabe klicev funkcij

**Klicanje zunanjih orodij**  
Klepetalni roboti so odlični pri odgovarjanju na vprašanja uporabnikov. S pomočjo klicev funkcij lahko klepetalni roboti uporabijo sporočila uporabnikov za opravljanje določenih nalog. Na primer, študent lahko klepetalnega robota prosi: "Pošlji e-pošto mojemu profesorju in mu sporoči, da potrebujem več pomoči pri tej temi." To lahko sproži klic funkcije `send_email(to: string, body: string)`

**Ustvarjanje API ali podatkovnih poizvedb**  
Uporabniki lahko poiščejo informacije z uporabo naravnega jezika, ki se nato pretvori v ustrezno poizvedbo ali API zahtevo. Primer tega je učitelj, ki vpraša: "Kateri študenti so oddali zadnjo nalogo?" in s tem sproži funkcijo z imenom `get_completed(student_name: string, assignment: int, current_status: string)`

**Ustvarjanje strukturiranih podatkov**  
Uporabniki lahko vzamejo besedilo ali CSV in uporabijo LLM za izluščitev pomembnih informacij. Na primer, študent lahko pretvori članek iz Wikipedije o mirovnih sporazumih v AI učne kartice. To lahko stori s funkcijo `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Ustvarjanje vašega prvega klica funkcije

Postopek ustvarjanja klica funkcije vključuje 3 glavne korake:
1. Klicanje Chat Completions API s seznamom vaših funkcij in sporočilom uporabnika
2. Preberite odgovor modela, da izvedete dejanje, npr. izvedete funkcijo ali API klic
3. Ponovno pokličite Chat Completions API z odgovorom vaše funkcije, da uporabite te informacije za ustvarjanje odgovora uporabniku.


![Potek klica funkcije](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.sl.png)


### Elementi klica funkcije

#### Uporabnikov vnos

Prvi korak je ustvariti sporočilo uporabnika. To lahko dinamično določite z vrednostjo iz besedilnega vnosa ali pa vrednost določite tukaj. Če prvič uporabljate API Chat Completions, moramo določiti `role` in `content` sporočila.

`role` je lahko `system` (določanje pravil), `assistant` (model) ali `user` (končni uporabnik). Za klic funkcije bomo to nastavili kot `user` in dodali primer vprašanja.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Ustvarjanje funkcij.

Nato bomo definirali funkcijo in njene parametre. Tukaj bomo uporabili samo eno funkcijo z imenom `search_courses`, lahko pa ustvarite več funkcij.

**Pomembno** : Funkcije so vključene v sistemsko sporočilo za LLM in bodo štele v skupno število razpoložljivih žetonov, ki jih imate na voljo.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definicije**

Struktura definicije funkcije ima več nivojev, vsak s svojimi lastnostmi. Tukaj je razčlenitev gnezdene strukture:

**Lastnosti funkcije na najvišji ravni:**

`name` - Ime funkcije, ki jo želimo poklicati.

`description` - Opis, kako funkcija deluje. Tukaj je pomembno, da ste natančni in jasni.

`parameters` - Seznam vrednosti in format, ki jih želite, da jih model ustvari v svojem odgovoru.

**Lastnosti objekta Parameters:**

`type` - Podatkovni tip objekta parameters (običajno "object")

`properties` - Seznam specifičnih vrednosti, ki jih bo model uporabil za svoj odgovor

**Lastnosti posameznega parametra:**

`name` - Posredno določeno s ključem lastnosti (npr. "role", "product", "level")

`type` - Podatkovni tip tega določenega parametra (npr. "string", "number", "boolean")

`description` - Opis določenega parametra

**Neobvezne lastnosti:**

`required` - Polje, ki navaja, kateri parametri so obvezni, da se klic funkcije lahko izvede


### Klic funkcije
Ko smo definirali funkcijo, jo moramo zdaj vključiti v klic Chat Completion API-ja. To naredimo tako, da v zahtevo dodamo `functions`. V tem primeru `functions=functions`.

Obstaja tudi možnost, da nastavite `function_call` na `auto`. To pomeni, da bomo LLM-ju prepustili odločitev, katero funkcijo naj pokliče glede na uporabnikovo sporočilo, namesto da bi to določili sami.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Zdaj si oglejmo odgovor in preverimo, kako je oblikovan:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Vidite lahko, da je ime funkcije poklicano in da je LLM iz uporabnikovega sporočila uspel najti podatke, ki ustrezajo argumentom funkcije.


## 3. Integracija klicev funkcij v aplikacijo.

Ko smo preizkusili formatiran odgovor iz LLM, ga lahko zdaj vključimo v aplikacijo.

### Upravljanje poteka

Da to vključimo v našo aplikacijo, sledimo naslednjim korakom:

Najprej izvedemo klic do OpenAI storitev in shranimo sporočilo v spremenljivko z imenom `response_message`.


In [ ]:
response_message = response.choices[0].message

Zdaj bomo definirali funkcijo, ki bo poklicala Microsoft Learn API za pridobitev seznama tečajev:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Kot dobra praksa bomo najprej preverili, ali želi model poklicati funkcijo. Nato bomo ustvarili eno izmed razpoložljivih funkcij in jo povezali s funkcijo, ki je bila poklicana.
Nato bomo argumente funkcije preslikali na argumente iz LLM.

Na koncu bomo sporočilu o klicu funkcije dodali še vrednosti, ki jih je vrnilo sporočilo `search_courses`. To LLM-ju zagotovi vse potrebne informacije,
da lahko uporabniku odgovori v naravnem jeziku.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Izziv s kodo

Odlično opravljeno! Če želite nadaljevati z učenjem o OpenAI Function Calling, lahko ustvarite: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst
 - Več parametrov funkcije, ki lahko pomagajo uporabnikom najti več tečajev. Razpoložljive parametre API-ja najdete tukaj:
 - Ustvarite še en klic funkcije, ki upošteva več informacij o uporabniku, na primer njegov materni jezik
 - Dodajte obravnavo napak, kadar klic funkcije in/ali API-ja ne vrne nobenega ustreznega tečaja



---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za strojno prevajanje [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da se zavedate, da lahko samodejni prevodi vsebujejo napake ali netočnosti. Izvirni dokument v svojem izvirnem jeziku naj velja za avtoritativni vir. Za ključne informacije priporočamo strokovni človeški prevod. Ne prevzemamo odgovornosti za morebitna nesporazume ali napačne razlage, ki bi nastale zaradi uporabe tega prevoda.
